In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os import path

url = r'https://www.pro-football-reference.com/years/2022/fantasy.htm'

In [2]:
# make an http request
r = requests.get(url)
print(r.status_code)

200


In [3]:
# instantiate soup object

soup = BeautifulSoup(r.content, 'html.parser')

# find all tables
player_table = soup.find_all('table')

print(len(player_table))

1


In [4]:
# read in table to dataframe

df = pd.read_html(str(player_table))[0]

df.head()

Unnamed: 0_level_0    Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk                Player                 Tm   
0                  1     Patrick Mahomes*+                KAN   
1                  2         Josh Jacobs*+                LVR   
2                  3  Christian McCaffrey*                2TM   
3                  4       Derrick Henry *                TEN   
4                  5    Justin Jefferson*+                MIN   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Games     Passing             ...  \
             FantPos                Age     G  GS     Cmp  Att   Yds  ...   
0                 QB                 27    17  17     435  648  5250  ...   
1                 RB                 24    17  17       0    0     0  ...   
2                 RB                 26    17  16       1    1    34  ...   
3                 RB                 28    16  16       2    2     4  ...   
4                 WR                 23    17  17       2    2    34  ...   

  Scoring           Fantasy                                           
       TD  2PM  2PP  FantPt    PPR   DKPt   FDPt  VBD PosRank OvRank  
0       4    1    2     416  417.4  435.4  428.9  136       1      1  
1      12  NaN  NaN     275  328.3  335.3  301.8  127       1      2  
2      13  NaN  NaN     271  356.4  362.4  313.9  123       2      3  
3      13  NaN  NaN     270  302.8  311.8  286.3  122       3      4  
4       9    1  NaN     241  368.7  371.7  304.7  119       1      5  

[5 rows x 33 columns]

In [5]:
# inspect columns

df.columns

MultiIndex([('Unnamed: 0_level_0',      'Rk'),
            ('Unnamed: 1_level_0',  'Player'),
            ('Unnamed: 2_level_0',      'Tm'),
            ('Unnamed: 3_level_0', 'FantPos'),
            ('Unnamed: 4_level_0',     'Age'),
            (             'Games',       'G'),
            (             'Games',      'GS'),
            (           'Passing',     'Cmp'),
            (           'Passing',     'Att'),
            (           'Passing',     'Yds'),
            (           'Passing',      'TD'),
            (           'Passing',     'Int'),
            (           'Rushing',     'Att'),
            (           'Rushing',     'Yds'),
            (           'Rushing',     'Y/A'),
            (           'Rushing',      'TD'),
            (         'Receiving',     'Tgt'),
            (         'Receiving',     'Rec'),
            (         'Receiving',     'Yds'),
            (         'Receiving',     'Y/R'),
            (         'Receiving',      'TD'),
            (

In [6]:
# drop unneeded columns

df = df.drop(columns = ['FantPt','PPR', 'DKPt', 'FDPt', 'VBD'], level=1)

df.head()

Unnamed: 0_level_0    Unnamed: 1_level_0 Unnamed: 2_level_0  \
                  Rk                Player                 Tm   
0                  1     Patrick Mahomes*+                KAN   
1                  2         Josh Jacobs*+                LVR   
2                  3  Christian McCaffrey*                2TM   
3                  4       Derrick Henry *                TEN   
4                  5    Justin Jefferson*+                MIN   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Games     Passing             ...  \
             FantPos                Age     G  GS     Cmp  Att   Yds  ...   
0                 QB                 27    17  17     435  648  5250  ...   
1                 RB                 24    17  17       0    0     0  ...   
2                 RB                 26    17  16       1    1    34  ...   
3                 RB                 28    16  16       2    2     4  ...   
4                 WR                 23    17  17       2    2    34  ...   

  Receiving           Fumbles    Scoring           Fantasy         
        Yds    Y/R TD     Fmb FL      TD  2PM  2PP PosRank OvRank  
0         6   6.00  0       5  0       4    1    2       1      1  
1       400   7.55  0       3  1      12  NaN  NaN       1      2  
2       741   8.72  5       1  0      13  NaN  NaN       2      3  
3       398  12.06  0       6  3      13  NaN  NaN       3      4  
4      1809  14.13  8       0  0       9    1  NaN       1      5  

[5 rows x 28 columns]

In [7]:
# flatten multi-level index

df.columns = ['_'.join(col) for col in df.columns] 

df.head()

,Unnamed: 0_level_0_Rk,Unnamed: 1_level_0_Player,Unnamed: 2_level_0_Tm,Unnamed: 3_level_0_FantPos,Unnamed: 4_level_0_Age,Games_G,Games_GS,Passing_Cmp,Passing_Att,Passing_Yds,...,Receiving_Yds,Receiving_Y/R,Receiving_TD,Fumbles_Fmb,Fumbles_FL,Scoring_TD,Scoring_2PM,Scoring_2PP,Fantasy_PosRank,Fantasy_OvRank
0,1,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,...,6,6.00,0,5,0,4,1,2,1,1
1,2,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,...,400,7.55,0,3,1,12,NaN,NaN,1,2
2,3,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,...,741,8.72,5,1,0,13,NaN,NaN,2,3
3,4,Derrick Henry *,TEN,RB,28,16,16,2,2,4,...,398,12.06,0,6,3,13,NaN,NaN,3,4
4,5,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,...,1809,14.13,8,0,0,9,1,NaN,1,5


In [8]:
# force cols to lower case

df.columns = [col.lower() for col in df.columns]

df.head()

,unnamed: 0_level_0_rk,unnamed: 1_level_0_player,unnamed: 2_level_0_tm,unnamed: 3_level_0_fantpos,unnamed: 4_level_0_age,games_g,games_gs,passing_cmp,passing_att,passing_yds,...,receiving_yds,receiving_y/r,receiving_td,fumbles_fmb,fumbles_fl,scoring_td,scoring_2pm,scoring_2pp,fantasy_posrank,fantasy_ovrank
0,1,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,...,6,6.00,0,5,0,4,1,2,1,1
1,2,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,...,400,7.55,0,3,1,12,NaN,NaN,1,2
2,3,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,...,741,8.72,5,1,0,13,NaN,NaN,2,3
3,4,Derrick Henry *,TEN,RB,28,16,16,2,2,4,...,398,12.06,0,6,3,13,NaN,NaN,3,4
4,5,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,...,1809,14.13,8,0,0,9,1,NaN,1,5


In [9]:
# clean up strings

df.columns = [i.split('_')[-1] if 'level' in str(i) else i for i in df.columns]

df.head()

,rk,player,tm,fantpos,age,games_g,games_gs,passing_cmp,passing_att,passing_yds,...,receiving_yds,receiving_y/r,receiving_td,fumbles_fmb,fumbles_fl,scoring_td,scoring_2pm,scoring_2pp,fantasy_posrank,fantasy_ovrank
0,1,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,...,6,6.00,0,5,0,4,1,2,1,1
1,2,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,...,400,7.55,0,3,1,12,NaN,NaN,1,2
2,3,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,...,741,8.72,5,1,0,13,NaN,NaN,2,3
3,4,Derrick Henry *,TEN,RB,28,16,16,2,2,4,...,398,12.06,0,6,3,13,NaN,NaN,3,4
4,5,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,...,1809,14.13,8,0,0,9,1,NaN,1,5


In [10]:
# rename columns

df = (df.rename(columns={
                'player': 'player_name',
                'fumbles_fmb' : 'fumbles',
                'fumbles_fl' : 'fumbles_lost',
                'games_g' : 'games',
                'games_gs' : 'games_started',
                'fantpos' : 'pos',
                'year_' : 'year'
                })
     )

df.head()

,rk,player_name,tm,pos,age,games,games_started,passing_cmp,passing_att,passing_yds,...,receiving_yds,receiving_y/r,receiving_td,fumbles,fumbles_lost,scoring_td,scoring_2pm,scoring_2pp,fantasy_posrank,fantasy_ovrank
0,1,Patrick Mahomes*+,KAN,QB,27,17,17,435,648,5250,...,6,6.00,0,5,0,4,1,2,1,1
1,2,Josh Jacobs*+,LVR,RB,24,17,17,0,0,0,...,400,7.55,0,3,1,12,NaN,NaN,1,2
2,3,Christian McCaffrey*,2TM,RB,26,17,16,1,1,34,...,741,8.72,5,1,0,13,NaN,NaN,2,3
3,4,Derrick Henry *,TEN,RB,28,16,16,2,2,4,...,398,12.06,0,6,3,13,NaN,NaN,3,4
4,5,Justin Jefferson*+,MIN,WR,23,17,17,2,2,34,...,1809,14.13,8,0,0,9,1,NaN,1,5


In [11]:
# remove player name characters

df['player_name'] = df['player_name'].str.split('*').str[0].str.split('+').str[0]

df['player_name'].unique()[:10]

array(['Patrick Mahomes', 'Josh Jacobs', 'Christian McCaffrey',
       'Derrick Henry ', 'Justin Jefferson', 'Austin Ekeler',
       'Josh Allen', 'Travis Kelce', 'Davante Adams', 'Nick Chubb'],
      dtype=object)